In [1]:
# Working with the Full Data (sex & Age) via Excel & Pickle files
# Finalized on 09/10/2019 @ 10:57 PM

In [2]:
import pandas as pd
import numpy as np
import string as str
import re

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from ipywidgets import widgets, interactive, Layout

In [3]:
pickle_file = 'Full_Data_df_w_Yearly_Totals.pkl' # 09/10/2019

In [4]:
Full_Data_df = pd.read_pickle(pickle_file)
Full_Data_df.rename(columns={'Unknown': 'Unknown_Sex'}, inplace=True)
Full_Data_df.rename(columns={'unknown_fm_age': 'Unknown_Age'}, inplace=True)

In [5]:
cols = list(Full_Data_df.columns.values)

In [6]:
#Swap two columnss Offense and Unknown_Sex
hdg1="Offense"
hdg2="Unknown_Sex"
indx1=cols.index(hdg1)
indx2=cols.index(hdg2)
print(indx1, indx2)
cols[indx1]=hdg2
cols[indx2]=hdg1

3 4


In [7]:
Full_Data_df=Full_Data_df.reindex(columns=cols)

In [8]:
offense_types=list(Full_Data_df["Offense"].unique())

In [9]:
min_year=1991
max_year=2017
quitcontrol="QUIT---All Done"
offtype = widgets.Dropdown(
    options=[quitcontrol]+offense_types+[quitcontrol],
    value='Yearly Totals All Crimes',
    description='Offense:',
)

start_year = widgets.BoundedFloatText(
    value=min_year,                        
    min=min_year,                          
    max=max_year-1,                          
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=max_year,                        
    min=min_year+1,                          
    max=max_year,                          
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

In [10]:
def plotit(offtype, start_year, end_year):
    df=Full_Data_df[Full_Data_df["Offense"]==offtype].copy()
    
    xlabsforbar=[]
    
    if end_year <= start_year:
        if start_year < max_year: 
            end_year=start_year+1
        else:
            start_year=end_year-1
        
    df=df[df["Year"]>=start_year]
    df=df[df["Year"]<=end_year]     
    #df.to_csv("offenders_by_age_"+offtype+".CSV")
    
    dfgrpbytype=df.groupby('Year').sum()
    dfbytype=dfgrpbytype.copy()
   
    indx=cols.index("Offense")
    dfsex=dfbytype.iloc[:,:indx]
    dfage=dfbytype.iloc[:,indx:]
   
    xvals=list(dfage.index)
    
    syr="{:.0f}".format(start_year)
    eyr="{:.0f}".format(end_year)

    #................................................................
    width=12
    height=8
    
    fig = plt.figure(figsize=(width,height))
    
    gs1 = GridSpec(4, 3)
    gs1.update(left=0.05, right=0.9, top=.9, bottom=0.01, wspace=0.35, hspace=.8)
    ax1 = plt.subplot(gs1[:-1, :])
    ax2 = plt.subplot(gs1[-1, :-1])
    ax3 = plt.subplot(gs1[-1, -1])

    ax1.set_xlabel("Years", fontsize=24)
    ax1.set_ylabel("# of Cases", fontsize=20)
    ax1.set_title("FBI Cases ("+syr+" - "+eyr+") for: "+offtype.title(), fontsize=20)
    ax1.grid(b=True, which='major', color='#666666', linestyle=':')
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    lgx=1.15
    lgy=1.02
    
    dfage=dfbytype.copy()
    xn=0
    sextotal=0
    agetotal=0
    for col in dfage.columns: 
        yvals=dfage[col]
        thistotal=yvals.sum()
        totalx="{:,}".format(thistotal)
        ctotal=totalx.rjust(16,".")
        
        lgtitl=col.replace("range","age").title()
        xlabsforbar.append(lgtitl)

        lgtitle=lgtitl.ljust(16,".")+ctotal
        if xn==2: lgtitle=lgtitle+"\n"
        if xn < 3: 
            ls="dashed" 
            dfage[col].plot(x=xvals,ax=ax1, label=lgtitle,linestyle=ls)
            sextotal=sextotal+thistotal
        else: 
            if xn < 8:
                ls="solid"
                ax1.plot(xvals, yvals, label=lgtitle,linestyle=ls)
                agetotal=agetotal+thistotal
            else:
                ls="dashdot"  #"dotted"
                ax1.plot(xvals, yvals, label=lgtitle,linestyle=ls)
                agetotal=agetotal+thistotal
        xn=xn+1
    
    cyrs=syr+"-"+eyr
    sexgt=("Grand Total By Sex: "+"{:,}".format(sextotal))
    agegt=("Grand Total By Age: "+"{:,}".format(agetotal))
    lghdg="\nNumbers shown represents totals for:\n\n> "+offtype.title()+"\n> for years: " \
          +cyrs+"\n\n"+sexgt+"\n"+agegt+"\n\n"
    ax1.legend(loc='upper center', bbox_to_anchor=(lgx, lgy),ncol=1, fancybox=True, shadow=True, title=lghdg)
    
    ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    ax1.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:.0f}".format(int(x))))
    ax1.grid(linestyle=':', linewidth='0.25', color='black')
    
    barwidth=0.4
    agegrp=list(dfage.sum(axis=0))
    xsexbar=xlabsforbar[:3]
    ysexbar=agegrp[:3]
    xagebar=xlabsforbar[3:]
    yagebar=agegrp[3:]
    
    ax2.bar(xagebar,yagebar, color="green", width=barwidth)
    ax2.set_xticklabels(xagebar, rotation=25, ha="right")
    ax2.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    ax2.grid(linestyle=':', linewidth='0.25', color='red')
    ax2.set_title("FBI Cases ("+syr+" - "+eyr+") By Age for: "+offtype.title())
    ax2.set_ylabel("# of Cases", fontsize=12)
      
    ax3.bar(xsexbar,ysexbar, color="red", width=barwidth)
    ax3.set_xticklabels(xsexbar, rotation=0)
    ax3.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    ax3.grid(linestyle=':', linewidth='0.25', color='green')
    title3="("+syr+" - "+eyr+") By Sex for: "+offtype.title()
    if len(title3)<=50:
        fsize=12
    else:
        if len(title3) < 60:
            fsize=10
        else:
            fsize=8
    ax3.set_title("("+syr+"-"+eyr+") By Sex for: "+offtype.title(), fontsize=fsize, fontname="Century", 
                                                fontweight=20, style="normal")
    ax3.tick_params(axis ='y', rotation =45) 

    plt.savefig('FBI_Data_for_Select_Yrs_between_1991-2017_'+offtype+'.png', bbox_inches='tight')
    plt.show()
    
    

In [11]:
def pre_plotit(offtype, start_year, end_year):
    if offtype==quitcontrol:
        print("\n\n\n All Done & Quitting .............. \n\n Good Bye! \n\n")
    else:
        plotit(offtype, start_year, end_year)


In [12]:
interactive(pre_plotit, offtype=offtype, start_year=start_year, end_year=end_year)

interactive(children=(Dropdown(description='Offense:', index=65, options=('QUIT---All Done', 'aggravated-assau…